# --------TAREA 1 CONFIGURACION DE ENTORNO Y PROCESO ETL INICIOS ------


**MAESTRIA EN CIBERSEGURIDAD**

**INTELIGENCIA DE NEGOCIOS**

**INTEGRANTES: JULIO AREVALO, EMILIO CORDOVA, NILSON ROMERO**

**2025**

## RESUMEN 
En este Notebook podrá encontrar el proceso completo de configuración de un entorno de análisis de datos y la ejecución de pipeline ETL. Se extraen datos de diferentes fuentes como PostgreSQL, CSV, JSON.
Se procede a cargar data Warehouse en PostgreSQL y se realiza una explotación y segmentación inicial de los datos utilizando librerías como Pandas en Python. 

# ----------IMPORTANCION DE LIBRERIAS CONEXION POSTGRESQL----------

Se inicia la actividad estableciendo la conexión a la base de datos de destino, esta DB funcionara como un **Data Warehouse**.
Utilizaremos como motor *DB* **PostgreSQL** esta base estará alojada en un contenedor **Docker**. Esto garantiza un entorno de desarrollo consistente, aislable y reproducible.
Se definen los parámetros de conexión y se creará el *motor* *(engine)* utilizando las librería *SQLalchemy*. El motor actuará como puente entre nuestra herramienta de análisis en Python y la base de datos.
En este proceso se ha implementado un bloque **try-except** para gestionar de forma precisa cualquier posible error durante el intento de conexión.

In [ ]:
# ----- IMPORTACION DE LIBRERIAS -----
# Se importan las librerías necesarias para la manipulación de datos y para la conexión con la base de datos 
# qué contendrá la data.

import pandas as pd
from sqlalchemy import create_engine # crea la conexion a la base de datos
import os # proporciona funciones para interactuar con el sistema operativo (manejar rutas de archivos)
import json # para trabajar con datos en formato JSON

# ----- CONEXION A LA BASE DE DATOS POSTGRESQL EN DOCKER -----
#------ Definición de los parametros de conexion a la base de datos -----
# Estos parámetros deben coincidir con los configurados en el contenedor Docker que ejecuta PostgreSQL.

db_user = 'postgres'
db_password = 'my_secret_password'
db_host = 'localhost'
db_port = '5432'
db_name = 'postgres'

#-----Creación del motor de conexión SQLalchemy-----
#se utiliza un bloque try-except para el manejo de errores.
#Si la conexión falla el programa informará del error en lugar de tenerse abruptamente.


print ("INICIANDO EL PROCESO DE CONEXION CON LA BESE DE DATOS")
try:
    engine = create_engine(f"postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")
    print("¡Conexión exitosa!")
except Exception as e:
    print(f"Error al conectar: {e}")    



INICIANDO EL PROCESO DE CONEXION CON LA BESE DE DATOS
¡Conexión exitosa!


En la salida nos indica que el proceso de conexion ha sido exitoso 


### CARGA MASIVA DE CSV A LA TABLA DE POSTGRESQL PROCESO ETL

En esta sección del código se podrá encontrar el núcleo del proceso **ETL** para inyectar la base de datos **postgreSQL**.
La consigna es consolidar múltiples archivos **CSV** relacionados en un único **data warehouse** y posteriormente se realizará el análisis de la información.

Se describe los pasos:

**Selección de la fuente EXTRACT:** Procedemos a definir una lista explícita de los archivos **CSV** que contienen las entidades del negocio las principales *(clientes, órdenes, productos)*

**Bucle de procesamiento:** se itegra sobre cada archivo de la lista para procesarlo de manera automatizada.

**Transformación ligera:** dentro del bucle se realiza una transformación básica para generar nombres de tabla limpios el lógicos a partir de los nombres de los archivos. ej. `olist_customers_dataset.csv` se convertirá en la tabla `customers`.

**Carga:** se utiliza la función **to_sql** de *Pandas* para cargar los datos de cada archivo en su tabla correspondiente dentro del progress SQL. Se ha configurado para reemplazar la tabla si ya existe.



In [ ]:
# Definimos la ruta donde se encuentran los archivos CSV descargados

data_path = r'C:\Users\j_dar\downloads\archive'

# SELECCIONAMOS LOS ARCHIVOS CSV
# Se crea una lista con los nombres de los archivos CSV que se van a cargar en la base de datos.
files_to_load_in_db = [
    'olist_customers_dataset.csv',
    'olist_orders_dataset.csv',
    'olist_order_items_dataset.csv',
    'olist_products_dataset.csv',
    'olist_sellers_dataset.csv'
]

#----- Blucle para cargar los archivos CSV en la base de datos -----
# INICIAMOS EL PROCESO DE CARGA MASIVA DE LOS ARCHIVOS CSV A LA BASE DE DATOS

print("\nIniciando carga de datos a PostgreSQL...")
for file in files_to_load_in_db:
    # El nombre de la tabla será el del archivo sin la extensión y prefijos
    table_name = file.replace('olist_', '').replace('_dataset.csv', '')
    df = pd.read_csv(os.path.join(data_path, file))
    df.to_sql(table_name, engine, if_exists='replace', index=False)
    print(f"Tabla '{table_name}' cargada con {len(df)} filas.")
print("Carga a PostgreSQL completada.")


Iniciando carga de datos a PostgreSQL...
Tabla 'customers' cargada con 99441 filas.
Tabla 'orders' cargada con 99441 filas.
Tabla 'order_items' cargada con 112650 filas.
Tabla 'products' cargada con 32951 filas.
Tabla 'sellers' cargada con 3095 filas.
Carga a PostgreSQL completada.


### GENERACION DE LOS DATAFRAME PRINCIPALES
En este proceso procederemos a realizar la creacion de los dataframes con la libreria pandas.

Se procede con la **extracción de datos desde tres fuentes distintas**, generando un **DataFrame** para cada una de ellas.
Este proceso de simulación va a reflejar un escenario empresarial *real* dónde la información reside en un sistema.

**DataFrame creados:**
**`df_orders` (Desde PostgreSQL ):** Se consultará la tabla de nuestro **DataWarehouse** esta presenta la fuente de datos principales ya consolidada y estructura.
**`df_order_reviews` (Desde CSV ):** Se leerá un archivo **CSV** externo, esto simula la ingesta de datos provenientes de un reporte externo o un sistema *Legacy*.
**`df_json` (Desde JSON):** Se simula una fuente `JSON` el proceso consistirá en convertir primero un archivo `csv` de referencia a un archivo `JSON` Y luego leerlo.


In [4]:
#----- Creacion del dataframe 1 desde PostGreSQL-----

print("1. Creando dataframe desde PostgreSQL tabla 'orders'")
query_orders = "SELECT * FROM orders;"
df_orders = pd.read_sql(query_orders, engine)

# ----- Creacion del dataframe 2 desde CSV -----

print("2. CREACION DE DATAFRAME DESDE CSV ('ORDER_REVIWS') ")
df_order_reviews = pd.read_csv(os.path.join(data_path, 'olist_order_reviews_dataset.csv'))

#----- creacion del dataframe 3 desde JSON -----

print("3. CREACION DE DATAFRAME DESDE JSON ('product_category_translation') ")

# convertir el csv a un archivo JSON PARA SIMULAR LA FUENTE
df_temp_json = pd.read_csv(os.path.join(data_path, 'product_category_name_translation.csv'))
json_path = os.path.join(data_path, 'categories.json')
df_temp_json.to_json(json_path, orient='records', lines=True)

# leer el archivo JSON
df_json = pd.read_json(json_path, lines=True)

print("LOS DATAFRAME PRINCIPALES HAN SIDO CREADOS EXITOSAMENTE")


1. Creando dataframe desde PostgreSQL tabla 'orders'
2. CREACION DE DATAFRAME DESDE CSV ('ORDER_REVIWS') 
3. CREACION DE DATAFRAME DESDE JSON ('product_category_translation') 
LOS DATAFRAME PRINCIPALES HAN SIDO CREADOS EXITOSAMENTE


# EXPLORACION Y FILTROS DEL DATAFRAME  DE POSTGRESQL DF_POSTGRES

Una vez que se ha procedido a cargar los **DataFrame** el siguiente paso es realizar el ** análisis exploratorio de datos EDA** este paso es fundamental para comprender la naturaleza de nuestra información antes de realizar otro tipo de análisis que sean más complejos.

Se inicia con él **DataFrame** `df_orders`, Que contiene la información de las órdenes de compra en nuestra base de datos **PostgreSQL**

**Se ha dividido el análisis en dos partes:**

**Exploración inicial:** Se utilizarán los métodos `.info()` y `.head()` de `Pandas` para responder 

**Filtrado y Segmentación:** Se aplicarán tres filtros específicos para segmentar los datos y responder a preguntas de negocio iniciales.

In [ ]:
# Exploracion de los dataframes creados
# Exploramos los dataframes creados para entender su estructura y contenido.

print("exploracion del postgreSQL df_postgres (orders)")
print(df_orders.info())
print("\nPrimeras cinco filas:")
display(df_orders.head())


exploracion del postgreSQL df_postgres (orders)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB
None

Primeras cinco filas:


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [11]:
# FILTRO PEDIDOS ENTREGADOS
entregados = df_orders[df_orders['order_status'] == 'delivered']
print(f"HAY {len(entregados)} PEDIDOS ENTREGADOS")

# PEDIDOS REALIZADOS EN EL 2017
pedidos_2017 = df_orders[pd.to_datetime(df_orders['order_purchase_timestamp']).dt.year == 2017]
print(f"hay {len(pedidos_2017)} pedidos realizados en el 2017")
pedidos_2017


HAY 96478 PEDIDOS ENTREGADOS
hay 45101 pedidos realizados en el 2017


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
5,a4591c265e18cb1dcee52889e2d8acc3,503740e9ca751ccdda7ba28e9ab8f608,delivered,2017-07-09 21:57:05,2017-07-09 22:10:13,2017-07-11 14:58:04,2017-07-26 10:57:55,2017-08-01 00:00:00
6,136cce7faa42fdb2cefd53fdc79a6098,ed0271e0b7da060a393796590e7b737a,invoiced,2017-04-11 12:22:08,2017-04-13 13:25:17,None,None,2017-05-09 00:00:00
7,6514b8ad8028c9f2cc2374ded245783f,9bdf08b4b3b52b5526ff42d37d47f222,delivered,2017-05-16 13:10:30,2017-05-16 13:22:11,2017-05-22 10:07:46,2017-05-26 12:55:51,2017-06-07 00:00:00
...,...,...,...,...,...,...,...,...
99433,9115830be804184b91f5c00f6f49f92d,da2124f134f5dfbce9d06f29bdb6c308,delivered,2017-10-04 19:57:37,2017-10-04 20:07:14,2017-10-05 16:52:52,2017-10-20 20:25:45,2017-11-07 00:00:00
99434,aa04ef5214580b06b10e2a378300db44,f01a6bfcc730456317e4081fe0c9940e,delivered,2017-01-27 00:30:03,2017-01-27 01:05:25,2017-01-30 11:40:16,2017-02-07 13:15:25,2017-03-17 00:00:00
99435,880675dff2150932f1601e1c07eadeeb,47cd45a6ac7b9fb16537df2ccffeb5ac,delivered,2017-02-23 09:05:12,2017-02-23 09:15:11,2017-03-01 10:22:52,2017-03-06 11:08:08,2017-03-22 00:00:00
99436,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28 00:00:00


In [10]:
# ----- PEDIDOS CANCELADOS O NO DISPONIBLES -----
cancelados = df_orders[df_orders['order_status'].isin(['canceled', 'unavailable'])]
print(f"HAY {len(cancelados)} PEDIDOS CANCELADOS O NO DISPONIBLES")
cancelados

HAY 1234 PEDIDOS CANCELADOS O NO DISPONIBLES


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
266,8e24261a7e58791d10cb1bf9da94df5c,64a254d30eed42cd0e6c36dddb88adf0,unavailable,2017-11-16 15:09:28,2017-11-16 15:26:57,None,None,2017-12-05 00:00:00
397,1b9ecfe83cdc259250e1a8aca174f0ad,6d6b50b66d79f80827b6d96751528d30,canceled,2018-08-04 14:29:27,2018-08-07 04:10:26,None,None,2018-08-14 00:00:00
586,c272bcd21c287498b4883c7512019702,9582c5bbecc65eb568e2c1d839b5cba1,unavailable,2018-01-31 11:31:37,2018-01-31 14:23:50,None,None,2018-02-16 00:00:00
613,714fb133a6730ab81fa1d3c1b2007291,e3fe72696c4713d64d3c10afe71e75ed,canceled,2018-01-26 21:34:08,2018-01-26 21:58:39,2018-01-29 22:33:25,None,2018-02-22 00:00:00
687,37553832a3a89c9b2db59701c357ca67,7607cd563696c27ede287e515812d528,unavailable,2017-08-14 17:38:02,2017-08-17 00:15:18,None,None,2017-09-05 00:00:00
...,...,...,...,...,...,...,...,...
99252,aaab15da689073f8f9aa978a390a69d1,df20748206e4b865b2f14a5eabbfcf34,unavailable,2018-01-16 14:27:59,2018-01-17 03:37:34,None,None,2018-02-06 00:00:00
99283,3a3cddda5a7c27851bd96c3313412840,0b0d6095c5555fe083844281f6b093bb,canceled,2018-08-31 16:13:44,None,None,None,2018-10-01 00:00:00
99347,a89abace0dcc01eeb267a9660b5ac126,2f0524a7b1b3845a1a57fcf3910c4333,canceled,2018-09-06 18:45:47,None,None,None,2018-09-27 00:00:00
99348,a69ba794cc7deb415c3e15a0a3877e69,726f0894b5becdf952ea537d5266e543,unavailable,2017-08-23 16:28:04,2017-08-28 15:44:47,None,None,2017-09-15 00:00:00


### EXPLORACION Y FILTROS DATAFRAME CSV

Esta vez enfocándonos en el DataFrame `df_order_reviews`, que fue cargado desde nuestra fuente CSV obligatoria.

Estos datos son **crítico para entender la satisfacción del cliente**, ya que contiene las puntuaciones y comentarios que los usuarios han dejado después de una compra.



**El proceso es el sigguiente:**

**Exploración de la Estructura de Datos** 

**Interpretación de Hallazgos Preliminares**

Se analizará la salida de la exploración para identificar puntos clave, como la cantidad de comentarios escritos versus reseñas sin texto, y la necesidad de convertir tipos de datos para análisis futuros.

In [12]:
print("----- exploracion de df_csv (reviews) -----")
print(df_order_reviews.info())
print("\nPrimeras cinco filas:")
display(df_order_reviews.head())

----- exploracion de df_csv (reviews) -----
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99224 entries, 0 to 99223
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   review_id                99224 non-null  object
 1   order_id                 99224 non-null  object
 2   review_score             99224 non-null  int64 
 3   review_comment_title     11568 non-null  object
 4   review_comment_message   40977 non-null  object
 5   review_creation_date     99224 non-null  object
 6   review_answer_timestamp  99224 non-null  object
dtypes: int64(1), object(6)
memory usage: 5.3+ MB
None

Primeras cinco filas:


,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


In [14]:
## ----- reseñas con una puntuación de cinco estrellas -----
resenias_5_estrellas = df_order_reviews[df_order_reviews['review_score'] == 5]
print(f"HAY {len(resenias_5_estrellas)} RESEÑAS CON 5 ESTRELLAS")
resenias_5_estrellas

HAY 57328 RESEÑAS CON 5 ESTRELLAS


,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53
6,07f9bee5d1b850860defd761afa7ff16,e48aa0d2dcec3a2e87348811bcfdf22b,5,NaN,NaN,2017-07-16 00:00:00,2017-07-18 19:30:34
...,...,...,...,...,...,...,...
99217,c6b270c61f67c9f7cb07d84ea8aeaf8b,48f7ee67313eda32bfcf5b9c1dd9522d,5,NaN,NaN,2017-12-13 00:00:00,2017-12-14 11:09:36
99218,af2dc0519de6e0720ef0c74292fb4114,d699c734a0b1c8111f2272a3f36d398c,5,NaN,NaN,2018-04-27 00:00:00,2018-04-30 01:18:57
99219,574ed12dd733e5fa530cfd4bbf39d7c9,2a8c23fee101d4d5662fa670396eb8da,5,NaN,NaN,2018-07-07 00:00:00,2018-07-14 17:18:30
99220,f3897127253a9592a73be9bdfdf4ed7a,22ec9f0669f784db00fa86d035cf8602,5,NaN,NaN,2017-12-09 00:00:00,2017-12-11 20:06:42


In [15]:
## ----- RESEÑAS CON COMENTARIOS -----
resenias_con_comentarios = df_order_reviews[df_order_reviews['review_comment_message'].notnull() & (df_order_reviews['review_comment_message'].str.strip() != '')]
print(f"HAY {len(resenias_con_comentarios)} RESEÑAS CON COMENTARIOS")
resenias_con_comentarios

HAY 40950 RESEÑAS CON COMENTARIOS


,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53
9,8670d52e15e00043ae7de4c01cc2fe06,b9bf720beb4ab3728760088589c62129,4,recomendo,aparelho eficiente. no site a marca do aparelh...,2018-05-22 00:00:00,2018-05-23 16:45:47
12,4b49719c8a200003f700d3d986ea1a19,9d6f15f95d01e79bd1349cc208361f09,4,NaN,"Mas um pouco ,travando...pelo valor ta Boa.\r\n",2018-02-16 00:00:00,2018-02-20 10:52:22
15,3948b09f7c818e2d86c9a546758b2335,e51478e7e277a83743b6f9991dbfa3fb,5,Super recomendo,"Vendedor confiável, produto ok e entrega antes...",2018-05-23 00:00:00,2018-05-24 03:00:01
...,...,...,...,...,...,...,...
99205,98fffa80dc9acbde7388bef1600f3b15,d398e9c82363c12527f71801bf0e6100,4,NaN,para este produto recebi de acordo com a compr...,2017-11-29 00:00:00,2017-11-30 15:52:51
99208,df5fae90e85354241d5d64a8955b2b09,509b86c65fe4e2ad5b96408cfef9755e,5,NaN,Entregou dentro do prazo. O produto chegou em ...,2018-02-07 00:00:00,2018-02-19 19:47:23
99215,a709d176f59bc3af77f4149c96bae357,d5cb12269711bd1eaf7eed8fd32a7c95,3,NaN,"O produto não foi enviado com NF, não existe v...",2018-05-19 00:00:00,2018-05-20 21:51:06
99221,b3de70c89b1510c4cd3d0649fd302472,55d4004744368f5571d1f590031933e4,5,NaN,"Excelente mochila, entrega super rápida. Super...",2018-03-22 00:00:00,2018-03-23 09:10:43


In [16]:
## RESEÑAS CON LA MINIMAPUNTUACIÓN 1 ESTRELLA
resenias_1_estrella = df_order_reviews[df_order_reviews['review_score'] == 1]
print(f"HAY {len(resenias_1_estrella)} RESEÑAS CON 1 ESTRELLA")
resenias_1_estrella

HAY 11424 RESEÑAS CON 1 ESTRELLA


,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
5,15197aa66ff4d0650b5434f1b46cda19,b18dcdf73be66366873cd26c5724d1dc,1,NaN,NaN,2018-04-13 00:00:00,2018-04-16 00:39:37
19,373cbeecea8286a2b66c97b1b157ec46,583174fbe37d3d5f0d6661be3aad1786,1,Não chegou meu produto,Péssimo,2018-08-15 00:00:00,2018-08-15 04:10:37
29,2c5e27fc178bde7ac173c9c62c31b070,0ce9a24111d850192a933fcaab6fbad3,1,NaN,Não gostei ! Comprei gato por lebre,2017-12-13 00:00:00,2017-12-16 07:14:07
32,58044bca115705a48fe0e00a21390c54,68e55ca79d04a79f20d4bfc0146f4b66,1,NaN,Sempre compro pela Internet e a entrega ocorre...,2018-04-08 00:00:00,2018-04-09 12:22:39
39,9fd59cd04b42f600df9f25e54082a8d1,3c314f50bc654f3c4e317b055681dff9,1,NaN,Nada de chegar o meu pedido.,2017-04-21 00:00:00,2017-04-23 05:37:03
...,...,...,...,...,...,...,...
99144,e5a991ae8286f513e512f0f9fd188f9b,fadb25c91522130892be8a5ddb3e139f,1,NaN,NaN,2018-04-14 00:00:00,2018-04-16 18:37:40
99155,2fc8295a24978092328d3a329d22940e,e809c167a9cfd31aee1293abe8995065,1,NaN,Eu recebi o seguinte email e preciso saber com...,2018-02-09 00:00:00,2018-02-13 13:03:24
99169,3450ec3bbabeb09a08b00fc72da87a30,c46f950813dd2dd9bab4188dceb83175,1,NaN,Boa tarde. \r\nNão recebo todos os produtos fa...,2017-12-08 00:00:00,2017-12-08 19:07:00
99184,6cf47345d15e054dd6df872e929bdb27,54e6829fe81bc86cf88b12e6d07ea298,1,NaN,NaN,2017-06-08 00:00:00,2017-06-08 22:52:39


### ANALISIS DE JSON 
Finalmente, se realiza el Análisis Exploratorio de Datos (EDA) para el DataFrame `df_json`. Este conjunto de datos fue cargado desde la **fuente JSON simulada** y contiene la traducción de los nombres de las categorías de productos del portugués al inglés.


In [17]:
# EXPLORACIÓN DE DF_JSON 
print("----- Exploración de df_json (categories_translation) -----")
print(df_json.info())
print("\nPrimeras cinco filas:")
display(df_json.head())



----- Exploración de df_json (categories_translation) -----
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 2 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   product_category_name          71 non-null     object
 1   product_category_name_english  71 non-null     object
dtypes: object(2)
memory usage: 1.2+ KB
None

Primeras cinco filas:


,product_category_name,product_category_name_english
0,beleza_saude,health_beauty
1,informatica_acessorios,computers_accessories
2,automotivo,auto
3,cama_mesa_banho,bed_bath_table
4,moveis_decoracao,furniture_decor


In [20]:
# ----- CATEGORIAS RELACIONADAS CON LA CAMA Y EL BAÑO -----
cama_bano = df_json[df_json['product_category_name_english'] == 'bed_bath_table']
print(f"HAY {len(cama_bano)} CATEGORIAS RELACIONADAS CON LA CAMA Y EL BAÑO")
cama_bano

HAY 1 CATEGORIAS RELACIONADAS CON LA CAMA Y EL BAÑO


,product_category_name,product_category_name_english
3,cama_mesa_banho,bed_bath_table


In [24]:
# CATEGORIAS RELACIONADAS CON LA PALABRA COMPUTADORA EN PORTUGUES
computadoras_pt = df_json[df_json['product_category_name'].str.contains('informatica', na=False)]
print(f"HAY {len(computadoras_pt)} CATEGORIAS RELACIONADAS CON LA PALABRA COMPUTADORA EN PORTUGUES")
computadoras_pt



HAY 1 CATEGORIAS RELACIONADAS CON LA PALABRA COMPUTADORA EN PORTUGUES


,product_category_name,product_category_name_english
1,informatica_acessorios,computers_accessories


In [25]:
# CATEGORIA REALCIONADA CON EL DEPORTE Y OCIO
deporte_ocio = df_json[df_json['product_category_name_english'] == 'sports_leisure']
print(f"HAY {len(deporte_ocio)} CATEGORIAS RELACIONADAS CON EL DEPORTE Y OCIO")
deporte_ocio

HAY 1 CATEGORIAS RELACIONADAS CON EL DEPORTE Y OCIO


,product_category_name,product_category_name_english
5,esporte_lazer,sports_leisure
